# Clinical Readmission ML – Modeling

## Problem Statement

The objective is to build a machine learning model to predict 
whether a patient will be readmitted to the hospital.

Target Variable:
- readmitted (yes / no)

Business Context:
In healthcare, missing a high-risk patient (false negative) 
can lead to increased medical complications and cost.

Primary Evaluation Metric:
- Recall (for class "yes")

Secondary Metric:
- F1-score

Approach:
1. Perform preprocessing (scaling + encoding)
2. Train baseline models
3. Apply cross-validation
4. Perform basic hyperparameter tuning
5. Select best model
6. Train final model
7. Save model for deployment

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import joblib

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import recall_score, f1_score

In [2]:
df = pd.read_csv("hospital_readmissions_cleaned.csv")
df.head()

,age,time_in_hospital,n_lab_procedures,n_procedures,n_medications,n_outpatient,n_inpatient,n_emergency,medical_specialty,diag_1,diag_2,diag_3,glucose_test,A1Ctest,change,diabetes_med,readmitted
0,[70-80),8,72,1,18.0,0.0,0.0,0.0,Missing,Circulatory,Respiratory,Other,no,no,no,yes,no
1,[70-80),3,34,2,13.0,0.0,0.0,0.0,Other,Other,Other,Other,no,no,no,yes,no
2,[50-60),5,45,0,18.0,0.0,0.0,0.0,Missing,Circulatory,Circulatory,Circulatory,no,no,yes,yes,yes
3,[70-80),2,36,0,12.0,0.0,0.0,0.0,Missing,Circulatory,Other,Diabetes,no,no,yes,yes,yes
4,[60-70),1,42,0,7.0,0.0,0.0,0.0,InternalMedicine,Other,Circulatory,Respiratory,no,no,no,yes,no


In [3]:
X = df.drop("readmitted", axis=1)
y = df["readmitted"]

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [5]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20000, 16), (5000, 16), (20000,), (5000,))

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

num_cols = X.select_dtypes(include="int64").columns
cat_cols = X.select_dtypes(include="object").columns


In [7]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ]
)

In [8]:
# Logistic Regression

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipe_lr = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LogisticRegression(max_iter=1000))
])

pipe_lr

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [9]:
pipe_lr.fit(X_train, y_train)


,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [10]:
y_pred_lr = pipe_lr.predict(X_test)
y_pred_lr

array(['yes', 'no', 'no', ..., 'yes', 'yes', 'yes'], dtype=object)

In [11]:
print("Logistic Recall:", recall_score(y_test, y_pred_lr, pos_label="yes"))
print("Logistic F1:", f1_score(y_test, y_pred_lr, pos_label="yes"))

Logistic Recall: 0.39302424500212674
Logistic F1: 0.44854368932038835


In [12]:
# Decision Tree

from sklearn.tree import DecisionTreeClassifier

pipe_dt = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", DecisionTreeClassifier(random_state=42))
])

pipe_dt

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [13]:
pipe_dt.fit(X_train, y_train)


,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [14]:
y_pred_dt = pipe_dt.predict(X_test)
y_pred_dt

array(['yes', 'no', 'no', ..., 'no', 'no', 'yes'], dtype=object)

In [15]:
print("DT Recall:", recall_score(y_test, y_pred_dt, pos_label="yes"))
print("DT F1:", f1_score(y_test, y_pred_dt, pos_label="yes"))

DT Recall: 0.5044661846022969
DT F1: 0.498110037799244


In [16]:
# Random Forest

from sklearn.ensemble import RandomForestClassifier

pipe_rf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestClassifier(random_state=42))
])

pipe_rf

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [17]:
pipe_rf.fit(X_train, y_train)


,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [18]:
y_pred_rf = pipe_rf.predict(X_test)
y_pred_rf

array(['yes', 'no', 'no', ..., 'no', 'no', 'yes'], dtype=object)

In [19]:
print("RF Recall:", recall_score(y_test, y_pred_rf, pos_label="yes"))
print("RF F1:", f1_score(y_test, y_pred_rf, pos_label="yes"))

RF Recall: 0.43428328370906
RF F1: 0.468886337543054


In [20]:
# Hyperparameter Tuning

param_grid_dt = {
    "model__max_depth": [None, 5, 10, 15, 20, 30],
    "model__min_samples_split": [2, 5, 10, 20],
    "model__min_samples_leaf": [1, 2, 5, 10],
    "model__criterion": ["gini", "entropy"]
}

In [21]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

skf

StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

In [22]:
# GridSearchCV

from sklearn.model_selection import GridSearchCV

grid_dt = GridSearchCV(
    estimator=pipe_dt,
    param_grid=param_grid_dt,
    cv=skf,
    scoring="recall_macro",   # safe for string labels
    n_jobs=-1
)

grid_dt

,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'model__criterion': ['gini', 'entropy'], 'model__max_depth': [None, 5, ...], 'model__min_samples_leaf': [1, 2, ...], 'model__min_samples_split': [2, 5, ...]}"
,scoring,'recall_macro'
,n_jobs,-1
,refit,True
,cv,StratifiedKFo... shuffle=True)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...)]"


In [23]:
grid_dt.fit(X_train, y_train)

,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'model__criterion': ['gini', 'entropy'], 'model__max_depth': [None, 5, ...], 'model__min_samples_leaf': [1, 2, ...], 'model__min_samples_split': [2, 5, ...]}"
,scoring,'recall_macro'
,n_jobs,-1
,refit,True
,cv,StratifiedKFo... shuffle=True)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...)]"


In [24]:
grid_dt.best_params_

{'model__criterion': 'entropy',
 'model__max_depth': 10,
 'model__min_samples_leaf': 5,
 'model__min_samples_split': 20}

In [25]:
grid_dt.best_score_

np.float64(0.54152693169872)

In [26]:
# Best Pipeline

best_pipeline = grid_dt.best_estimator_

best_pipeline

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


#### Final Evaluation on Test Set

In [27]:
from sklearn.metrics import classification_report, recall_score, f1_score

y_pred_final = best_pipeline.predict(X_test)
y_pred_final

array(['yes', 'no', 'no', ..., 'yes', 'yes', 'yes'], dtype=object)

In [28]:
print(classification_report(y_test, y_pred_final))


              precision    recall  f1-score   support

          no       0.56      0.51      0.54      2649
         yes       0.50      0.55      0.52      2351

    accuracy                           0.53      5000
   macro avg       0.53      0.53      0.53      5000
weighted avg       0.53      0.53      0.53      5000



In [29]:
print("Final Recall:", recall_score(y_test, y_pred_final, pos_label="yes"))
print("Final F1:", f1_score(y_test, y_pred_final, pos_label="yes"))

Final Recall: 0.5478519778817524
Final F1: 0.5227272727272727


In [30]:
# Save the Model

import joblib
joblib.dump(best_pipeline, "hospital_model.pkl")

['hospital_model.pkl']

### Final Model Summary

- After tuning the Decision Tree using cross-validation, the model achieved a recall of about 0.55 for patients who were readmitted.

- This means the model is able to correctly identify around 55% of patients who are likely to be readmitted.

- Since recall is the main focus in this project (because missing high-risk patients is more serious), the tuned Decision Tree model is selected as the final model for deployment.